1. gRNA calculator

In [8]:
from Bio import SeqIO
from Bio import Seq
import csv
import numpy
import pandas as pd

def pam_calc2(chromosome_file, gene_start, gene_end, insert, direction, num_rows):
    """
    The purpose of this function is to return n number (inputted) of optimal PAMs 
    within a gene given a certain insertion site. Inputs needed are a genbank
    file of the chromosome, the start and ending positions of the gene on the
    chromosome, the insertion site (with relation to the entire chromosome), and
    n number of optimal PAMs desired in the output. 
    """
    # load chromosome and deal with forward case 
    load = SeqIO.read(chromosome_file, 'genbank')
    chromosome = load.seq
    chromosome_length = len(chromosome)
    fwd_gene = chromosome[gene_start - 2:gene_end + 2]
    fwd_insert = insert
    
    # deal with reverse case
    rev_chromosome = chromosome.reverse_complement()
    rev_gene_start = chromosome_length - gene_start
    rev_gene_end = chromosome_length - gene_end
    rev_gene = rev_chromosome[rev_gene_end - 2:rev_gene_start + 2]
    rev_insert = chromosome_length - fwd_insert

    # make a table where PAMs will be entered
    pams = pd.DataFrame(columns = ['PAM', 'position', 'insert dist', 'gRNA',
    'gRNA start', 'gRNA end'])
    if (direction == "+"):
        # forward case 
        for x in range(len(fwd_gene)): 
            # define current position in relation to chromosome 
            current_chromosome_pos = x + (gene_start - 2)
            # calculate distance from current position to insert 
            current_distance = current_chromosome_pos - fwd_insert
            # define current search (3 nucleotides)
            current_search = fwd_gene[x:x+3] 
            # check to see if the current_search is a PAM 
            if((current_search == 'AGG') or (current_search == 'TGG') or (current_search == 'GGG') or (current_search == 'CGG')):
                # make gRNA molecule
                gRNA_start = current_chromosome_pos - 20
                gRNA_end = current_chromosome_pos
                gRNA = chromosome[gRNA_start:gRNA_end]
                # append PAM to dataframe
                pams = pams.append({'PAM': str(current_search), 'position': 
                    current_chromosome_pos, 'insert dist': current_distance,
                    'gRNA': str(gRNA), 'gRNA start': gRNA_start, 'gRNA end':
                    gRNA_end}, ignore_index=True)
    if(direction == "-"):
        # reverse case 
        for x in range (len(rev_gene)): 
            # define current position in relation to chromosme 
            current_chromosome_pos = x + (rev_gene_start - 2)
            # calculate distance from current position to insert 
            current_distance = current_chromosome_pos - rev_insert
            # define current search (3 nucleotides)
            current_search = rev_gene[x:x+3] 
            # check to see if the current_search is a PAM 
            if((current_search == 'CCA') or (current_search == 'CCT') or (current_search == 'CCG') or (current_search == 'CCC')):
                # make gRNA molecule
                gRNA_start = current_chromosome_pos - 20
                gRNA_end = current_chromosome_pos
                gRNA = chromosome[gRNA_start:gRNA_end]
                # append PAM to dataframe
                pams = pams.append({'PAM': str(current_search), 'position': 
                    current_chromosome_pos, 'insert dist': current_distance, 
                    'gRNA': str(gRNA), 'gRNA start': gRNA_start, 'gRNA end':
                    gRNA_end}, ignore_index=True) 
    

    # sort dataframe based on distance between insert and PAMs
    pams = pams.sort_values(by = 'insert dist') 

    # return num_rows of 'pams' dataframe 
    # return pams.head(num_rows)
    # return entire table
    return pams


gene: ADE2 (-)

position: 565619

n: 10

In [9]:
from Bio import SeqIO
from Bio import Seq
import csv
import numpy
import pandas as pd
import os

file = 'chromosome 15.flat'          # chromosome file associated with ADE2
start = 564475                       # start of gene on chromosome
end = 566191                         # end of gene on chromosome
insert = 565619                      # insert site 
num_rows = 10                        # 'n'
direction = "-"                      # direction of transcribed gene

'''
IMPORTANT: what strand do we want our gRNA to be based off (5' -> 3' or 3' -> 5'?).
This gene is transcribed on the 3' -> 5' (-)
'''

table = pam_calc2(file, start, end, insert, direction, num_rows) 
print(table)


    PAM position insert dist                  gRNA gRNA start gRNA end
0   CCT   526927        1255  CCGTAAACGAATACGTAAGT     526907   526927
1   CCA   526931        1259  AAACGAATACGTAAGTACGA     526911   526931
2   CCA   526949        1277  GAAATTACAAAGCATGTCTG     526929   526949
3   CCA   526956        1284  CAAAGCATGTCTGATAAGTT     526936   526956
4   CCT   526970        1298  TAAGTTCACATCAACCTCCA     526950   526970
..  ...      ...         ...                   ...        ...      ...
57  CCC   528274        2602  CTGCATCGTTGTCATCTGTG     528254   528274
58  CCG   528275        2603  TGCATCGTTGTCATCTGTGA     528255   528275
59  CCT   528331        2659  ATAGACGAATTCCACCACCA     528311   528331
60  CCA   528343        2671  CACCACCAGGTTTTAGTCAA     528323   528343
61  CCG   528353        2681  TTTTAGTCAAAATATATTAA     528333   528353

[62 rows x 6 columns]


From here, we have our table of possible PAMS. We need to automate the selection process of the most
optimal PAM/gRNA pair.

In [10]:
from Bio import SeqIO
from Bio import Seq
import csv
import numpy
import pandas as pd
import os

# first sort list based on absolute value of distances from insert 
table['insert dist'] = table['insert dist'].abs()
table = table.sort_values(by = 'insert dist') 
print(table)
# let's assume most optimal gRNA has the closest insert distance (first row in df)
selection = table.iloc[0]
print(selection)


    PAM position insert dist                  gRNA gRNA start gRNA end
0   CCT   526927        1255  CCGTAAACGAATACGTAAGT     526907   526927
1   CCA   526931        1259  AAACGAATACGTAAGTACGA     526911   526931
2   CCA   526949        1277  GAAATTACAAAGCATGTCTG     526929   526949
3   CCA   526956        1284  CAAAGCATGTCTGATAAGTT     526936   526956
4   CCT   526970        1298  TAAGTTCACATCAACCTCCA     526950   526970
..  ...      ...         ...                   ...        ...      ...
57  CCC   528274        2602  CTGCATCGTTGTCATCTGTG     528254   528274
58  CCG   528275        2603  TGCATCGTTGTCATCTGTGA     528255   528275
59  CCT   528331        2659  ATAGACGAATTCCACCACCA     528311   528331
60  CCA   528343        2671  CACCACCAGGTTTTAGTCAA     528323   528343
61  CCG   528353        2681  TTTTAGTCAAAATATATTAA     528333   528353

[62 rows x 6 columns]
PAM                             CCT
position                     526927
insert dist                    1255
gRNA           CC

This is weird... why is the closest PAM 1255 base pairs away? is it possible to have regions of the 

the gene that don't have any PAMs? 

Also, do we want the gRNA to bind to the strand of the gene or be identical to the strand of the gene?

Going under the assumption that the gRNA needs to be on the same strand as the gene. 

2. Template DNA 

In [ ]:
from Bio import SeqIO
from Bio import Seq
import csv
import numpy
import pandas as pd
import os

# import LOV domain via biopython 
'''
Where do I get the LOV domain from? Need a .gb file or something along those lines. 
'''

3. 31-mer Barcode

In [ ]:
from Bio import SeqIO
from Bio import Seq
import csv
import numpy
import pandas as pd
import os

'''
What is this exactly?
'''